In the first cell, we define our query function. This is to be copied and pasted each time we want to query science windows. 

In [77]:
import time
import astroquery.heasarc
from astropy.coordinates import SkyCoord
import astropy.units as u

Heasarc = astroquery.heasarc.Heasarc()

def get_scw_list(ra_obj, dec_obj,radius,start_date,end_date ):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            catalog = 'intscw', # 'mission' has been deprecated, use catalog instead
            time = start_date + " .. " + end_date,
            good_isgri = ">1000",
        )
    R.sort('scw_id') # changed SCW_ID to scw_id
    R['scw_id', 'scw_ver', 'start_date', 'end_date', 'ra', 'dec'].pprint()
    return R['scw_id'], R['scw_ver']  # changed SCW_VER to scw_ver


assert astroquery.__version__ >= '0.4.2.dev6611'
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

columns = Heasarc.list_columns(catalog_name='intscw')
columns.sort('name')
columns[:].pprint(align='<')

      name      ...  unit 
--------------- ... ------
data_in_heasarc ...       
data_size       ... byte  
dec             ... degree
end_date        ... mjd   
good_isgri      ... s     
good_jemx1      ... s     
good_jemx2      ... s     
good_omc        ... s     
good_picsit     ... s     
good_spi        ... s     
obs_id          ...       
obs_type        ...       
pi_name         ...       
ra              ... degree
scw_id          ...       
scw_type        ...       
scw_ver         ...       
start_date      ... mjd   
status          ...       


In this second cell, define our source or region to query.
What we want to do for Jupiter is find Jupiter's time-dependent position and query that for a given search radius and time interval. We use astroquery's JPL Horizons module.

In [78]:
from astroquery.jplhorizons import Horizons

epochs = {'start':'2004-01-01', 'stop':'2004-02-01', 'step':'1d'}

# Jupiter's ID in Horizons is 599
# We use @earth as we want geocentric coordinates
jupiter = Horizons(id='599', location='@0',epochs=epochs) # 599 is Jupiter's ID in this module
eph = jupiter.ephemerides()
print(eph)

ra_values = eph['RA']  # in degrees
dec_values = eph['DEC']  # in degrees
print(eph[['datetime_str', 'RA', 'DEC']])

  targetname     datetime_str   datetime_jd ... alpha_true  PABLon  PABLat
     ---             ---             d      ...    deg       deg     deg  
------------- ----------------- ----------- ... ---------- -------- ------
Jupiter (599) 2004-Jan-01 00:00   2453005.5 ...     0.0176 159.0596 1.1123
Jupiter (599) 2004-Jan-02 00:00   2453006.5 ...     0.0175 159.1366 1.1132
Jupiter (599) 2004-Jan-03 00:00   2453007.5 ...     0.0175 159.2135 1.1141
Jupiter (599) 2004-Jan-04 00:00   2453008.5 ...     0.0175 159.2904  1.115
Jupiter (599) 2004-Jan-05 00:00   2453009.5 ...     0.0175 159.3673 1.1159
Jupiter (599) 2004-Jan-06 00:00   2453010.5 ...     0.0175 159.4442 1.1168
Jupiter (599) 2004-Jan-07 00:00   2453011.5 ...     0.0174  159.521 1.1178
Jupiter (599) 2004-Jan-08 00:00   2453012.5 ...     0.0174 159.5979 1.1186
Jupiter (599) 2004-Jan-09 00:00   2453013.5 ...     0.0174 159.6748 1.1195
Jupiter (599) 2004-Jan-10 00:00   2453014.5 ...     0.0174 159.7516 1.1204
          ...            

Error due to seperation between INTEGRAL and Earth.

In [79]:
D = 150e3 # max distance between INTEGRAL and Earth (~ 150 000 km)
r = 4 * 149597870.7 # min distance between Jupiter and Earth is ~ 4 AU 

error = D/r * 206265/60 # convert to arcmin
print("Angular error is ", error, " arcmin") # below 1', insignificant (FWHM is 12' for INTEGRAL)

Angular error is  0.8617477267341882  arcmin


Retrieve the science windows. Make sure to choose ones that are of 'pointing' type.

In [ ]:
scw_ids = []
scw_versions = []

# Loop through the ephemeris
for row in eph:
    ra = float(row['RA'])
    dec = float(row['DEC'])
    radius = 8.0 # search radius in degrees
    scw_id, scw_ver = get_scw_list(ra, dec, radius, '2004-01-01T00:00:00', '2004-02-01T00:00:00')
    scw_ids.extend(scw_id)
    scw_versions.extend(scw_ver)

# filter by type
scw_ids = scw_ids[scw_ids['scw_type']=='pointing']
scw_versions = scw_versions[scw_versions['scw_type']=='pointing']

print("Found science windows (pointing):", scw_ids)

   scw_id    scw_ver    start_date        end_date         ra      dec   
                            d                d            deg      deg   
------------ ------- ---------------- ---------------- --------- --------
007600040171     001 52788.5195513002 52788.5478730735 166.73833 12.52730
008200200021     001 52806.2710191345 52806.2830446035 167.28118 10.47886
026601100010     001 53358.8256557167 53358.8325191468 156.62770 10.60867
026601100021     001 53358.8325191468 53358.8338501662 156.45857 10.20109
026601110010     001 53358.8338501662 53358.8443131356 156.11153  9.37400
026601110022     001 53358.8443131356 53358.8546372161 156.18837  9.36603
026601110031     001 53358.8546372161 53358.8737460243 167.04653  5.20160
026700000210     001 53359.4028435524 53359.4254130089 154.57039  5.04156
026700000221     001 53359.4254130089  53359.426443102 154.56520  5.03179
026700000230     001  53359.426443102 53359.4493713549 154.56081  5.02164
         ...     ...              ... 

We then pick any science window(s) we want. 

In [62]:
scw_pick =  [(s+"."+v).strip() for s,v in zip(scw_ids,scw_versions)]
scw_pick = scw_pick[0]
print(scw_pick)

007600040171.001


Before moving on to using this query for our ODA, we need to input our token. 

In [64]:
token = ''

import getpass
token = getpass.getpass('Insert the token')

We also add logging to help visualize the process.

In [65]:
import logging
logging.getLogger().setLevel(logging.INFO) # WARNING, INFO or DEBUG
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Now let us look at an example.

In [66]:
from oda_api.api import DispatcherAPI

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data")

par_dict = {
"E1_keV": "15",
"E2_keV": "30",
"detection_threshold": "7",
"instrument": "isgri",
"osa_version": "OSA10.2",
"product": "isgri_image",
"product_type": "Real",
"scw_list": scw_pick,
'token': token
}
# "integral_data_rights": "all-private" had to be removed as didn't have the rights

data_collection = disp.get_product(**par_dict)
data_collection.show()
data_collection.mosaic_image_0_mosaic.show()

print(data_collection.mosaic_image_0_mosaic.show_meta())


found token in TokenLocation.FILE_CUR_DIR your token payload: {
    "email": "hucklethums@icloud.com",
    "exp": 1741191208,
    "name": "interstellxr",
    "roles": "authenticated user, public-pool-hpc",
    "sub": "hucklethums@icloud.com"
}
found token in TokenLocation.FILE_CUR_DIR your token payload: {
    "email": "hucklethums@icloud.com",
    "exp": 1741191208,
    "name": "interstellxr",
    "roles": "authenticated user, public-pool-hpc",
    "sub": "hucklethums@icloud.com"
}
token expires in 219.2 h
token expires in 219.2 h
discovered token in environment
discovered token in environment
please beware that by default, in a typical setup, oda_api will not output much. To learn how to increase the verbosity, please refer to the documentation: https://oda-api.readthedocs.io/en/latest/user_guide/ScienceWindowList.html?highlight=logging#Let's-get-some-logging . 
To disable this message you can pass `.get_product(..., silent=True)`
please beware that by default, in a typical setup, od

KeyboardInterrupt: 

Finally, let us image our results.

In [ ]:
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np

# from oda_api.plot_tools import OdaImage

fits_image = data_collection.mosaic_image_0_mosaic

with fits.open(fits_image) as hdul:
    image_data = hdul[0].data 

# Plot the image
plt.figure(figsize=(10, 8))
plt.imshow(image_data, cmap='inferno', origin='lower', norm=plt.Normalize(vmin=np.percentile(image_data, 5), vmax=np.percentile(image_data, 99)))  
plt.colorbar(label="Counts")
plt.title("INTEGRAL/ISGRI X-ray Image")
plt.xlabel("X Pixel")
plt.ylabel("Y Pixel")
plt.show()